# Analytics Goal: Review Features Across Time

In [1]:
# sc.install_pypi_package("plotly")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
102,application_1638590815625_0103,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import time
start = time.time()#put this at the begining
data_path = "s3://msds694-project-zhipeng/data"
rdd = sc.textFile(data_path)
header=rdd.take(1)
rdd=rdd.filter(lambda x:x != header[0])
rdd=rdd.map(lambda x:x.split('\t'))
rdd.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PythonRDD[3] at RDD at PythonRDD.scala:53

In [3]:
def mean(x):
    return sum(x)/len(x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:


# focus on the review which total vote >0, for those total vote=0 we dont know whether these are helpful
helpful_rate=rdd.map(lambda x:(int(x[8]),int(x[9]),x[-1][:4])).filter(lambda x:x[1] != 0).filter(lambda x:x[2]> '2005')
helpful_rate=helpful_rate.map(lambda x:(x[0]/x[1],x[2]))
helpful_rate=helpful_rate.map(lambda x:(x[1],(x[0])))
helpful_rate_countbykey=helpful_rate.sortByKey().countByKey()
helpful_rate=helpful_rate.groupByKey().mapValues(lambda x:mean(x)).sortByKey()
# total review number
total_review=rdd.map(lambda x:x[-1][:4]).filter(lambda x:x > '2005').sortBy(lambda x:x[0],ascending=False).countByValue()



print('second' , time.time() - start) #put this at the end we’d like to test/ in the end


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

second 124.22804427146912

In [5]:
helpful_rate_countbykey = sorted(helpful_rate_countbykey.items(),key=lambda x: x[1],reverse=False)
helpful_rate_countbykey = sc.parallelize(helpful_rate_countbykey)
helpful_rate_countbykey = helpful_rate_countbykey.toDF()
helpful_rate_countbykey.createOrReplaceTempView("helpful_rate_countbykey_view")
# helpful_rate_countbykey.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
total_review = sorted(total_review.items(),key=lambda x: x[1],reverse=False)
total_review = sc.parallelize(total_review)
total_review = total_review.toDF()
total_review.createOrReplaceTempView("total_review_view")
# total_review.show()
helpful_rate=helpful_rate.toDF()
helpful_rate.createOrReplaceTempView("helpful_rate_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
%%sql -o helpful_rate_countbykey
SELECT * FROM helpful_rate_countbykey_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [8]:
%%sql -o total_review
SELECT * FROM total_review_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [9]:
%%sql -o helpful_rate
SELECT * FROM helpful_rate_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [10]:
%%local 
click_rate=total_review
click_rate['_2']=helpful_rate_countbykey['_2']/total_review['_2']
click_rate

Output()

In [11]:
%%local
### click rate=  vote !=0 /total count 
### helpful rate = helpful vote/total vote

import plotly
import plotly.graph_objects as go

x = helpful_rate['_1']
y1 =helpful_rate['_2']
y2= click_rate['_2']
fig = go.Figure()
# Use the hovertext kw argument for hover text
fig.add_trace(go.Scatter(x=x, y=y1, name='Helpful Rate',line = dict(color='rgb(255,192,203)', width=3),mode='lines'))
fig.add_trace(go.Scatter(x=x, y=y2, name='Click Rate',line = dict(color='rgb(49,130,189)', width=3),mode='lines'))


fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)
annotations = []

#annotation
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Helpful and Click Rate',
                              font=dict(family='Arial',
                                        size=25,
                                        color='rgb(37,37,37)'),
                              showarrow=False))

annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Year',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))
# Legend
fig.update_layout(legend=dict(
    orientation="v",
    yanchor="top",
    y=1.02,
    xanchor="right",
    x=1.3
))
fig.update_layout(annotations=annotations)

fig.show()

In [12]:
# review count by category with different year

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
start_rate_rdd=rdd.map(lambda x:((x[-1][:7],x[6]),(int(x[7])))).groupByKey().mapValues(lambda x:mean(x))
start_rate_rdd=start_rate_rdd.map(lambda x:(x[0][0],x[0][1],x[1])).sortBy(lambda x:(x[0],x[1])).filter(lambda x:x[0][:4]>'2005')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
start_rate = start_rate_rdd.toDF()
start_rate.createOrReplaceTempView("start_rate_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
%%sql -o start_rate
SELECT * FROM start_rate_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [16]:
%%local
import plotly.graph_objects as go



fig = go.Figure()
colors = ['rgb(255,192,203)','rgb(49,130,189)', 'rgb(210,105,30)', 'rgb(46,139,87)',  'rgb(189,189,189)']
label=['Ebook','Music','Software','Video_Download','Video_purchase']
i=0

for index,data in  start_rate.groupby('_2'):
    color=colors[i]
    i+=1
    year=data['_1']
    fig.add_trace(go.Scatter(x=year, y=data['_3'], name=index[8:],line = dict(color=color, width=3),mode='lines'))

annotations = []

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)
annotations = []

# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Star Rating for Each Category',
                              font=dict(family='Arial',
                                        size=25,
                                        color='rgb(37,37,37)'),
                              showarrow=False))

annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Year',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))
# Legend
fig.update_layout(legend=dict(
    orientation="v",
    yanchor="top",
    y=1.02,
    xanchor="right",
    x=1.3
))
fig.update_layout(annotations=annotations)

fig.show()

In [17]:
review_count_month=rdd.map(lambda x:x[-1]).filter(lambda x:x[:4]>'2005')
review_count_month=review_count_month.map(lambda x:x[:7])
review_count_month=review_count_month.sortBy(lambda x:x[0],ascending=False).countByValue()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
review_count_month = sorted(review_count_month.items(),key=lambda x: x[0],reverse=False)
review_count_month = sc.parallelize(review_count_month)
review_count_month = review_count_month.toDF()
review_count_month.createOrReplaceTempView("review_count_month_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
%%sql -o review_count_month
SELECT * FROM review_count_month_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [20]:
%%local

import plotly
import plotly.graph_objects as go

x = review_count_month['_1']
y=review_count_month['_2']
max_value=review_count_month[review_count_month['_1']=='2013-01-01']
fig = go.Figure()
# Use the hovertext kw argument for hover text
fig.add_trace(go.Scatter(x=x, y=y, name='Review Count',line = dict(color='rgb(49,130,189)', width=3),mode='lines'))
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)
annotations = []

#annotation
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Review Count by Month',
                              font=dict(family='Arial',
                                        size=25,
                                        color='rgb(37,37,37)'),
                              showarrow=False))

annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Year',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

annotations.append(dict(xref='paper', yref='paper', x=0.75, y=1.01,
                              xanchor='center', yanchor='top',
                              text='Max Count at 2013-01',
                              font=dict(family='Arial',
                                        size=12,
                                        color='black'),
                              showarrow=False))
# Legend
fig.update_layout(legend=dict(
    orientation="v",
    yanchor="top",
    y=1.02,
    xanchor="right",
    x=1.3
))

fig.update_layout(annotations=annotations)

fig.show()

In [21]:
start_rate_total=rdd.map(lambda x:(x[-1][:7],(int(x[7])))).groupByKey().mapValues(lambda x:mean(x))
start_rate_total=start_rate_total.filter(lambda x:x[0][:4]>'2005').sortBy(lambda x:(x[0],x[1]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
start_rate_total = start_rate_total.toDF()
start_rate_total.createOrReplaceTempView("start_rate_total_view")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
%%sql -o start_rate_total
SELECT * FROM start_rate_total_view

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [27]:
%%local

import plotly
import plotly.graph_objects as go

x = start_rate_total['_1']
y=start_rate_total['_2']
# max_value=review_count_month[review_count_month['_1']=='2013-01-01']
fig = go.Figure()
# Use the hovertext kw argument for hover text
fig.add_trace(go.Scatter(x=x, y=y, name='Star Rating',line = dict(color='rgb(49,130,189)', width=3),mode='lines'))
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)
annotations = []

#annotation
annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Star Rating By Month',
                              font=dict(family='Arial',
                                        size=25,
                                        color='rgb(37,37,37)'),
                              showarrow=False))

annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Year',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

# annotations.append(dict(xref='paper', yref='paper', x=0.75, y=1.01,
#                               xanchor='center', yanchor='top',
#                               text='Max Count at 2013-01',
#                               font=dict(family='Arial',
#                                         size=12,
#                                         color='black'),
#                               showarrow=False))
# Legend
fig.update_layout(legend=dict(
    orientation="v",
    yanchor="top",
    y=1.02,
    xanchor="right",
    x=1.3
))

fig.update_layout(annotations=annotations)

fig.show()


In [25]:
print('Total time', time.time() - start)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total time 215.20348691940308

Total time 215.203

Effeciency test

In [26]:
# rdd.cache()
# rdd.count()
# import time
# start = time.time()#put this at the begining

# # focus on the review which total vote >0, for those total vote=0 we dont know whether these are helpful
# helpful_rate=rdd.map(lambda x:(int(x[8]),int(x[9]),x[-1][:4])).filter(lambda x:x[1] != 0).filter(lambda x:x[2]> '2005')
# helpful_rate=helpful_rate.map(lambda x:(x[0]/x[1],x[2]))
# helpful_rate=helpful_rate.map(lambda x:(x[1],(x[0])))
# helpful_rate_countbykey=helpful_rate.sortByKey().countByKey()
# helpful_rate=helpful_rate.groupByKey().mapValues(lambda x:mean(x)).sortByKey()
# # total review number
# total_review=rdd.map(lambda x:x[-1][:4]).filter(lambda x:x > '2005').sortBy(lambda x:x[0],ascending=False).countByValue()



# print('Second', time.time() - start) #put this at the end we’d like to test/ in the end

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…